In [16]:
import pandas as pd
from tqdm import tqdm
import networkx as nx
import json
from networkx.algorithms.tree.recognition import *

In [2]:
pages_df = pd.read_csv('pages.csv')
pages_df.head()

/Users/khonzoda/opt/miniconda3/envs/networks-class-env/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,page_id,page_namespace,page_title,page_restrictions,page_counter,page_is_redirect,page_is_new,page_random,page_touched,page_latest,page_len
0,6,0,AmericanSamoa,NaN,0,1,0,0.387835,20070727152503,133452270,48
1,8,0,AppliedEthics,NaN,0,1,0,0.279509,20070727152503,133452279,49
2,10,0,AccessibleComputing,NaN,0,1,0,0.331671,20070727152503,133452289,57
3,12,0,Anarchism,NaN,5252,0,0,0.786172,20080103215623,181964860,67759
4,13,0,AfghanistanHistory,NaN,5,1,0,0.062150,20070727152503,74466652,57


In [3]:
pages_dct = pages_df.to_dict()
pages_dct.keys()

dict_keys(['page_id', 'page_namespace', 'page_title', 'page_restrictions', 'page_counter', 'page_is_redirect', 'page_is_new', 'page_random', 'page_touched', 'page_latest', 'page_len'])

In [4]:
page_id_to_title = {}
page_title_to_id = {}
duplicates = []
for i in tqdm(range(len(pages_dct['page_id']))):
    
    page_id = str(pages_dct['page_id'][i])
    page_title = pages_dct['page_title'][i]
    page_namespace = pages_dct['page_namespace'][i]
    
    page_id_to_title[page_id] = page_title
    if page_title not in page_title_to_id:
        page_title_to_id[page_title] = {str(page_namespace): page_id}
    elif str(page_namespace) not in page_title_to_id[page_title]:
        page_title_to_id[page_title][str(page_namespace)] = page_id
    else:
        duplicates.append((page_title, page_title_to_id[page_title], page_id))

len(page_id_to_title), len(page_title_to_id)


def get_page_id_from_title(title, namespace, mapping=page_title_to_id):
    if title in page_title_to_id and namespace in page_title_to_id[title]:
        return page_title_to_id[title][namespace]
    else:
        return None
    
def get_page_title_from_id(id_, mapping=page_id_to_title):
    if id_ in page_id_to_title:
        return page_id_to_title[id_]
    else:
        return None

100%|███████████████████████████| 11372877/11372877 [00:49<00:00, 231583.81it/s]


In [5]:
len(duplicates)

6

In [6]:
with open('docmap') as f:
    text = f.read().split('\n')

In [81]:
cs_article_ids_to_titles = {}
cs_article_titles_to_ids = {}
for line in text:
    id_ = line.split('\t')[0]
    title = line.split('\t')[1]
    cs_article_ids_to_titles[int(id_)] = title
    cs_article_titles_to_ids[title] = int(id_)

In [82]:
len(cs_article_titles_to_ids), len(cs_article_ids_to_titles)

(14468, 14468)

In [8]:
get_page_title_from_id('14105005')

'Contents'

In [12]:
with open('wiki_category_bsf_tree.json') as f:
    bfs_category_tree = json.load(f)

In [14]:
G = nx.DiGraph()

In [15]:
for n1, n2 in bfs_category_tree:
    G.add_edge(n1, n2)

In [18]:
is_tree(G)

True

In [86]:
leaves = [n for n, d in dict(G.out_degree()).items() if d==0]
leaves[0]

11278636

In [87]:
len(leaves)

23378

In [89]:
len(cs_article_ids_to_titles.keys())

14468

In [90]:
cs_article_leaves = set(cs_article_ids_to_titles.keys()).intersection(set(leaves))
len(cs_article_leaves)

12924

In [39]:
cs_pages_for_computation = list(pd.read_csv('Semantic_Networks_Comparison-main/names.csv', header=None)[0])
len(cs_pages_for_computation)

168

In [63]:
cs_page_ids_for_computation = [get_page_id_from_title(elt.replace(' ', '_'),'0') for elt in cs_pages_for_computation]
cs_page_ids_for_computation = [int(elt) if elt is not None else 'None' for elt in cs_page_ids_for_computation]
# len(cs_page_ids_for_computation)

In [64]:
cs_page_ids_for_computation[0]

4964625

In [65]:
cs_pages_for_computation[-4], cs_pages_for_computation[-5]

('FishEye', 'Swatting')

In [66]:
[elt for elt in list(cs_article_titles_to_ids.keys()) if 'Fish' in elt]

['FishBase',
 'Fisher_information',
 'Fred_Fish',
 'Year_of_the_Fish',
 'Fisher-Yates_shuffle',
 'FishEye']

In [50]:
cs_article_titles_to_ids['FishEye']

'17420845'

In [83]:
cs_page_ids_from_docmap = [cs_article_titles_to_ids[elt] for elt in cs_pages_for_computation]

In [84]:
zipped = zip(cs_page_ids_for_computation, cs_page_ids_from_docmap, cs_pages_for_computation)
pages_to_consider_by_title = {}
pages_to_consider_by_id = {}
for id1, id2, title in zipped:
    if id1 != id2:
        print(id1, id2)
    else:
        pages_to_consider_by_title[title] = id1
        pages_to_consider_by_id[id1] = title

326734 18935353
None 15709175
None 17420845
20430 18933632
30713 18934325
14086808 18987740


In [68]:
len(pages_to_consider_by_title), len(pages_to_consider_by_id)

(162, 162)

In [74]:
pages_that_are_leaves = set(leaves).intersection(set(pages_to_consider_by_id.keys()))
len(pages_that_are_leaves)

159

In [76]:
with open('updates_page_names.json', 'w') as f:
    json.dump({pages_to_consider_by_id[k]: k for k in pages_that_are_leaves}, f)

In [93]:
source = 14105005

In [94]:
path_lengths = [nx.shortest_path_length(G, source=source, target=n) for n in cs_article_leaves]
max(path_lengths), min(path_lengths)

(9, 3)

In [95]:
path_lengths = [nx.shortest_path_length(G, source=source, target=n) for n in leaves]
max(path_lengths), min(path_lengths)

(11, 2)

In [100]:
all_paths[0]

[14105005, 14104879, 7345184, 690747, 990361, 776364, 2392067]

In [103]:
all_paths = []
cs_articles_set = set(cs_article_leaves)
for n in tqdm(cs_article_leaves):
    paths = nx.all_simple_paths(G, source=source, target=n)
    all_paths += [path for path in paths if len(set(path[:-1]).intersection(cs_articles_set))==0]

100%|█████████████████████████████████████| 12924/12924 [15:34<00:00, 13.83it/s]


In [104]:
len(all_paths)

12924

In [106]:
edge_strings = []
for path in all_paths:
    pairs = zip(path, path[1:])
    edge_strings += ['{}_{}'.format(parent, child) for parent, child in pairs]

In [107]:
unique_edge_strings = list(set(edge_strings))
len(unique_edge_strings)

14868

In [108]:
trimmed_edges_list = []
for string in unique_edge_strings:
    elts = string.split('_')
    parent = int(elts[0])
    child = int(elts[1])
    trimmed_edges_list.append(((parent, child)))

In [111]:
G_trimmed = nx.DiGraph()

In [112]:
for n1, n2 in tqdm(trimmed_edges_list):
    G_trimmed.add_edge(n1, n2)

100%|█████████████████████████████████| 14868/14868 [00:00<00:00, 122606.61it/s]


In [113]:
is_tree(G)

True

In [114]:
def get_leaves(graph):
    return [n for n, d in dict(graph.out_degree()).items() if d==0]

In [115]:
leaves = get_leaves(G_trimmed)
leaves[0]

30657

In [116]:
len(leaves)

12924

In [117]:
len(cs_article_ids_to_titles.keys())

14468

In [118]:
cs_article_leaves = set(cs_article_ids_to_titles.keys()).intersection(set(leaves))
len(cs_article_leaves)

12924

In [119]:
source = 14105005

In [120]:
path_lengths = [nx.shortest_path_length(G_trimmed, source=source, target=n) for n in cs_article_leaves]
max(path_lengths), min(path_lengths)

(9, 3)

In [121]:
def compute_branching_factor(graph):
    out_degrees = [d for _, d in dict(graph.out_degree()).items()]
    return sum(out_degrees)/len(out_degrees)

In [122]:
compute_branching_factor(G_trimmed)

0.9999327459815724